In [1]:
import sqlite3
import pandas as pd
import MeCab
import ipadic
import re
import sys
sys.path.append("../chat")
import openai
import os

In [2]:
import chat
from chat.db import query
from chat.entry_points.load_keywords import condition
from chat.chunk import morphological_analysis
from chat.prompt import question2keywords, keywords2simirality, load_content

In [3]:
question = "ふくおかフィナンシャルグループと横浜銀行は、どういう関係がありますか"
kws = question2keywords(question)
sims = keywords2simirality(kws)

query: reading cache


In [4]:
load_content(sims["chunk_id"][:5])

query: reading cache


['以上本件に関するご照会は下記までお願いいたします。ふくおかフィナンシャルグループ経営企画部財務Ｇ電話092-723-2502。',
 '以上本件に関するご照会は下記までお願いいたします。ふくおかフィナンシャルグループ経営企画部財務Ｇ電話092-723-2502。',
 '以上本件に関するご照会は下記までお願いいたします。ふくおかフィナンシャルグループ経営企画部財務Ｇ電話092-723-2502。',
 '以上本件に関するご照会は下記までお願いいたします。ふくおかフィナンシャルグループ経営企画部財務Ｇ電話092-723-2502。',
 '以上本件に関するご照会は下記までお願いいたします。ふくおかフィナンシャルグループ経営企画部財務Ｇ電話092-723-2502。']

問題は何かと言うと、questionと関係ないチャンクを拾ってしまっている。今、重要度は、各ドキュメント内での各単語のtfidfの和（質問側のtfidfは考えてないので）。で、tfは、ドキュメントの長さが分母に入っているので、長い文章では、この評価は相対的に不利なはず。

tfをなんで使いたいかというと、単語がたくさん出てきたら、それに関係する文章だろうから。なんだけど、ターゲットの単語が複数回でる可能性は低い。また、今、ドキュメントの長さは最大で300に制限しているから、ドキュメントの長さで正規化する意味が薄い、というか、ほとんどの文章は大体、300文字の中、一部に極端に短い文章が存在するという構造になっていると考えられるので、そういった短い文章をブーストする方向に効いていると考えられる。これは、やりたいことの逆。

tfidfではなくて、ドキュメントの正規化操作を無効にしたような重要度でソートしてみる。